# Maximizing flow through network

In this example, our task will be to maximize flow through defined network, which is perfekt example of linear problem.

We have a network (represented as directed graph). Graph has **Source** - source of a medium (it can be for example gas, water or electricity), **Sink** and some other nodes in between. All edges between source and sink has defined **maximal capacity** (numbers on the edges).

![Directed graph - flow](data/flow_graph.drawio.svg)

What is the maximum of medium, we can transmit through this network?

Lets label edges and nodes:

![Directed graph - flow](data/flow_graph2.drawio.svg)

And use linear program to solve this problem.

In [20]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

In [21]:
# Empty list of constraints
constraints = []
# variables, for each edge one variable
edge = cp.Variable(15)
# maximal capacity for each edge
maximal_capacity = np.array([13.0, 14, 10, 4, 10, 7, 6, 8, 5, 10, 2, 10, 7, 11, 8])

In [22]:
# negative flow does not make sence in this case
constraints.append(edge >= 0)
# capacity constraints (flow can not exceed capacity of edge)
constraints.append(edge <= maximal_capacity)
# flow constraints (what flows into node has to be the same as what flows out of node)
constraints.extend([
    edge[0] == edge[4],
    edge[1] == edge[5] + edge[6],
    edge[2] == edge[7],
    edge[3] == edge[8],
    edge[4] + edge[5] == edge[9] + edge[11],
    edge[9] == edge[10] + edge[12],
    edge[6] + edge[7] + edge[10] == edge[13],
    edge[11] == edge[14],
])
# cost function is sum of all edges ending in sink
cost = edge[8] + edge[12] + edge[13] + edge[14]
# objective is maximize
objective = cp.Maximize(cost)
# define LP problem
problem = cp.Problem(objective, constraints)

In [23]:
# solve LP problem with ECOS solver
problem.solve(verbose=True, solver=cp.ECOS)


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -1.794e+01  -1.742e+02  +2e+02  7e-04  4e-09  1e+00  5e+00    ---    ---    1  1  - |  -  - 
 1  -2.167e+01  -4.230e+01  +2e+01  9e-05  5e-08  1e-01  7e-01  0.8712  4e-03   0  0  0 |  0  0
 2  -2.954e+01  -3.605e+01  +7e+00  3e-05  1e-07  6e-02  2e-01  0.7109  4e-02   0  0  0 |  0  0
 3  -2.973e+01  -3.066e+01  +9e-01  4e-06  2e-08  3e-02  3e-02  0.9890  1e-01   0  0  0 |  0  0
 4  -3.000e+01  -3.001e+01  +1e-02  4e-08  4e-09  3e-04  4e-04  0.9885  1e-04   0  0  0 |  0  0
 5  -3.000e+01  -3.000e+01  +1e-04  5e-10  8e-11  4e-06  4e-06  0.9890  1e-04   1  0  0 |  0  0
 6  -3.000e+01  -3.000e+01  +1e-06  5e-12  2e-12  4e-08  4e-08  0.9890  1e-04   1  0  0 |  0  0
 7  -3.000e+01  -3.000e+01  +1e-08  6e-14  5e-13  5e-10  5e-10  0.9890  1e-04   1  0  0 |  0  0

OPTIMAL (within feastol=5.5e-13, reltol=4.9e-

29.99999999621742

In [24]:
# print results
print(f"Maximized volume of medium: {problem.value}")
print("Flow in each edge:")
for i in range(len(maximal_capacity)):
    print(
        f"edge {i} - capacity"
        + f" {np.around(edge[i].value, decimals=3)}"
        + f" / {maximal_capacity[i]}"
    )

Maximized volume of medium: 29.99999999621742
Flow in each edge:
edge 0 - capacity 9.036 / 13.0
edge 1 - capacity 10.542 / 14.0
edge 2 - capacity 6.422 / 10.0
edge 3 - capacity 4.0 / 4.0
edge 4 - capacity 9.036 / 10.0
edge 5 - capacity 6.215 / 7.0
edge 6 - capacity 4.326 / 6.0
edge 7 - capacity 6.422 / 8.0
edge 8 - capacity 4.0 / 5.0
edge 9 - capacity 7.251 / 10.0
edge 10 - capacity 0.251 / 2.0
edge 11 - capacity 8.0 / 10.0
edge 12 - capacity 7.0 / 7.0
edge 13 - capacity 11.0 / 11.0
edge 14 - capacity 8.0 / 8.0
